In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import the dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [3]:
# Read the CSV into a DataFrame
df_sales = pd.read_csv(
    "/content/drive/My Drive/colab_files/sales.csv")

In [4]:
df_sales.head()

,PageVisitsCat1,PageVisitDurationCat1,PageVisitsCat2,PageVisitDurationCat2,PageVisitsCat3,PageVisitDurationCat3,BounceRates,ExitRates,PageValues,HowCloseToSpecialDay,Month,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,Feb,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,Feb,Returning_Visitor,False,False
2,0.0,0.0,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,Feb,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,Feb,Returning_Visitor,True,False
4,0.0,0.0,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,Feb,Returning_Visitor,False,False


In [5]:
# List the columns
df_sales.columns

Index(['PageVisitsCat1', 'PageVisitDurationCat1', 'PageVisitsCat2',
       'PageVisitDurationCat2', 'PageVisitsCat3', 'PageVisitDurationCat3',
       'BounceRates', 'ExitRates', 'PageValues', 'HowCloseToSpecialDay',
       'Month', 'VisitorType', 'Weekend', 'Revenue'],
      dtype='object')

In [6]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PageVisitsCat1         12283 non-null  float64
 1   PageVisitDurationCat1  12283 non-null  float64
 2   PageVisitsCat2         12283 non-null  float64
 3   PageVisitDurationCat2  12283 non-null  float64
 4   PageVisitsCat3         12283 non-null  float64
 5   PageVisitDurationCat3  12283 non-null  float64
 6   BounceRates            12283 non-null  float64
 7   ExitRates              12283 non-null  float64
 8   PageValues             12283 non-null  float64
 9   HowCloseToSpecialDay   12283 non-null  float64
 10  Month                  12283 non-null  object 
 11  VisitorType            12283 non-null  object 
 12  Weekend                12283 non-null  bool   
 13  Revenue                12283 non-null  bool   
dtypes: bool(2), float64(10), object(2)
memory usage: 1.1+ 

### Description of columns:

**PageVisitsCatX:** Number of pages visited by a visitor. Cat1 is administrative, Cat2 is informational, and Cat3 is product related.

**PageVisitDurationCatX:** How long a visitor stayed on the category of pages.

**BounceRates:** Percentage of visitors who landed and exited a page.

**ExitRates:** Percentage of visitors who left the site from that page.

**PageValues:** A measurement of a page's contribution to a sale.

**HowCloseToSpecialDay:** How close is browsing date to a special day or a holiday, such as Valentine's day? Higher numbers are closer.

**Month:** Month of visit.

**VisitorType:** Is the visitor a new or returning visitor?

**Weekend:** Did the visit occur on a weekend day?

**Revenue:** Did the visit conclude in a sale?

## Perform Data Analysis

In [7]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [11]:
df = spark.read.csv(SparkFiles.get("/content/drive/My Drive/colab_files/sales.csv"), header=True)
df.show()

+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+----------+--------------------+-----+-----------------+-------+-------+
|PageVisitsCat1|PageVisitDurationCat1|PageVisitsCat2|PageVisitDurationCat2|PageVisitsCat3|PageVisitDurationCat3|         BounceRates|           ExitRates|PageValues|HowCloseToSpecialDay|Month|      VisitorType|Weekend|Revenue|
+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+----------+--------------------+-----+-----------------+-------+-------+
|           0.0|                  0.0|           0.0|                  0.0|           1.0|                  0.0|                 0.2|                 0.2|       0.0|                 0.0|  Feb|Returning_Visitor|  False|  False|
|           0.0|                  0.0|           0.0|                  0.0|           2.0|  

In [12]:
# Create a temporary view of the DataFrame.
df.createOrReplaceTempView('customer')

In [16]:
# Retrieve the number visits for each month.
sql = """
select month, count(visitortype) from customer
group by 1
order by 2 desc
"""
spark.sql(sql).show()

+-----+------------------+
|month|count(visitortype)|
+-----+------------------+
|  May|              3357|
|  Nov|              2995|
|  Mar|              1884|
|  Dec|              1727|
|  Oct|               549|
|  Sep|               448|
|  Aug|               433|
|  Jul|               431|
| June|               288|
|  Feb|               171|
+-----+------------------+



In [21]:
# Retrieve the number of visits that resulted in a purchase or not.
sql = """
select count(revenue) from customer

"""
spark.sql(sql).show()

+--------------+
|count(revenue)|
+--------------+
|         12283|
+--------------+



In [36]:
# What percentage of visits resulted in a purchase?
sql = """
with zag as(
select count(revenue) purchased, (select count(revenue) from customer)as total from customer
where revenue = 'True')
select CONCAT(round((purchased/total)*100,2), '%') AS percnt_purchased_visits from zag

"""
spark.sql(sql).show()

+-----------------------+
|percnt_purchased_visits|
+-----------------------+
|                 15.53%|
+-----------------------+



In [37]:
# Filter the DataFrame for all purchases.
sql= """
select * from customer
where revenue = True

"""
spark.sql(sql).show()


+--------------+---------------------+--------------+---------------------+--------------+---------------------+-----------+--------------------+------------------+--------------------+-----+-----------------+-------+-------+
|PageVisitsCat1|PageVisitDurationCat1|PageVisitsCat2|PageVisitDurationCat2|PageVisitsCat3|PageVisitDurationCat3|BounceRates|           ExitRates|        PageValues|HowCloseToSpecialDay|Month|      VisitorType|Weekend|Revenue|
+--------------+---------------------+--------------+---------------------+--------------+---------------------+-----------+--------------------+------------------+--------------------+-----+-----------------+-------+-------+
|           3.0|          87.83333333|           0.0|                  0.0|          27.0|    798.3333332999999|        0.0|         0.012643678|        22.9160357|                 0.8|  Feb|Returning_Visitor|  False|   True|
|          10.0|          1005.666667|           0.0|                  0.0|          36.0|      

In [44]:
# What is the number of visits for each month that resulted in a purchase?

# Which month had the greatest number of visits where a purchase was made?

sql = """
select month, count(visitortype), dense_rank() over(order by count(visitortype) desc) as rnk from customer
where revenue = 'True'
group by 1
order by 2 desc
"""
spark.sql(sql).show()

print(f"Month with greatest number of visits where a purchase was made is NOVEMBER with 760 visits")


+-----+------------------+---+
|month|count(visitortype)|rnk|
+-----+------------------+---+
|  Nov|               760|  1|
|  May|               365|  2|
|  Dec|               216|  3|
|  Mar|               192|  4|
|  Oct|               115|  5|
|  Sep|                86|  6|
|  Aug|                76|  7|
|  Jul|                66|  8|
| June|                29|  9|
|  Feb|                 3| 10|
+-----+------------------+---+

Month with greatest number of visits where a purchase was made is NOVEMBER with 760 visits


In [47]:
# How many visits were from returning visitors?
# How many visits were from new visitors?
sql ="""
select count(*) New_Visitor, (select count(*) from customer
where visitortype = 'Returning_Visitor') AS Returning_Visitor  from customer
where visitortype = 'New_Visitor'

"""
spark.sql(sql).show()

+-----------+-----------------+
|New_Visitor|Returning_Visitor|
+-----------+-----------------+
|       1694|            10504|
+-----------+-----------------+



In [51]:
# How many visits took place on a weekday? And, on the weekend?
sql = """
select count(*) weekday_visits, (select count(*) from customer where Weekend = "True") AS weekend_visits
From customer
where weekend = 'False'

"""
spark.sql(sql).show()

sql = """

select count(*) total, Weekend from customer
group by 2

"""
spark.sql(sql).show()

+--------------+--------------+
|weekday_visits|weekend_visits|
+--------------+--------------+
|          9421|          2862|
+--------------+--------------+

+-----+-------+
|total|Weekend|
+-----+-------+
| 9421|  False|
| 2862|   True|
+-----+-------+



## Perform Logistic Regression 

In [58]:
df.show()

+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+----------+--------------------+-----+-----------------+-------+-------+
|PageVisitsCat1|PageVisitDurationCat1|PageVisitsCat2|PageVisitDurationCat2|PageVisitsCat3|PageVisitDurationCat3|         BounceRates|           ExitRates|PageValues|HowCloseToSpecialDay|Month|      VisitorType|Weekend|Revenue|
+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+----------+--------------------+-----+-----------------+-------+-------+
|           0.0|                  0.0|           0.0|                  0.0|           1.0|                  0.0|                 0.2|                 0.2|       0.0|                 0.0|  Feb|Returning_Visitor|  False|  False|
|           0.0|                  0.0|           0.0|                  0.0|           2.0|  

In [63]:
df2 = df.show()

+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+----------+--------------------+-----+-----------------+-------+-------+
|PageVisitsCat1|PageVisitDurationCat1|PageVisitsCat2|PageVisitDurationCat2|PageVisitsCat3|PageVisitDurationCat3|         BounceRates|           ExitRates|PageValues|HowCloseToSpecialDay|Month|      VisitorType|Weekend|Revenue|
+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+----------+--------------------+-----+-----------------+-------+-------+
|           0.0|                  0.0|           0.0|                  0.0|           1.0|                  0.0|                 0.2|                 0.2|       0.0|                 0.0|  Feb|Returning_Visitor|  False|  False|
|           0.0|                  0.0|           0.0|                  0.0|           2.0|  

In [67]:
# Convert categorical variables to binary variables 
df_one =pd.get_dummies(df_sales)
df_one

,PageVisitsCat1,PageVisitDurationCat1,PageVisitsCat2,PageVisitDurationCat2,PageVisitsCat3,PageVisitDurationCat3,BounceRates,ExitRates,PageValues,HowCloseToSpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,0.0,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
3,0.0,0.0,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
4,0.0,0.0,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12278,3.0,145.0,0.0,0.0,53.0,1783.791667,0.007143,0.029031,12.241717,0.0,...,0,0,0,0,0,0,0,0,0,1
12279,0.0,0.0,0.0,0.0,5.0,465.750000,0.000000,0.021333,0.000000,0.0,...,0,0,0,0,1,0,0,0,0,1
12280,0.0,0.0,0.0,0.0,6.0,184.250000,0.083333,0.086667,0.000000,0.0,...,0,0,0,0,1,0,0,0,0,1
12281,4.0,75.0,0.0,0.0,15.0,346.000000,0.000000,0.021053,0.000000,0.0,...,0,0,0,0,1,0,0,0,0,1


In [68]:
# List the columns
for col in df_one:
  print(col)

PageVisitsCat1
PageVisitDurationCat1
PageVisitsCat2
PageVisitDurationCat2
PageVisitsCat3
PageVisitDurationCat3
BounceRates
ExitRates
PageValues
HowCloseToSpecialDay
Weekend
Revenue
Month_Aug
Month_Dec
Month_Feb
Month_Jul
Month_June
Month_Mar
Month_May
Month_Nov
Month_Oct
Month_Sep
VisitorType_New_Visitor
VisitorType_Other
VisitorType_Returning_Visitor


In [69]:
# Separate the features and target variables.
y=df_one["Revenue"]
X=df_one.drop(columns="Revenue")

In [70]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)

In [71]:
print(X_train.shape)
print(X_test.shape)
print(X.shape)

(9212, 24)
(3071, 24)
(12283, 24)


In [72]:
# Instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [73]:
# Train the model
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [75]:
# Use the testing data to make predictions.
predictions = classifier.predict(X_test)

pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
3066,False,False
3067,False,False
3068,False,True
3069,False,False


In [76]:
# Calculate the accuracy. 
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.882544507164568
Testing Data Score: 0.8922175187235428


In [77]:
# Print the confusion matrix.

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[2536,   58],
       [ 273,  204]])

In [78]:
# Print a classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.90      0.98      0.94      2594
        True       0.78      0.43      0.55       477

    accuracy                           0.89      3071
   macro avg       0.84      0.70      0.75      3071
weighted avg       0.88      0.89      0.88      3071



## Overall Model Performance
----

- **Question:** Can the logistic regression model predict whether visiting a website will result in a purchase? 

- **Answer:**  Yes, We had an accuracy of 89% which can improve with tuning hyperparameters  
